In [1]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [222]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'remm_base_year.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [207]:
parcel_data = pd.read_csv(r"E:\Tasks\Replace_HAFB_Parcels\Base_Year_Data\parcels.csv")
parcel_geog = pd.read_csv(r"E:\Tasks\Replace_HAFB_Parcels\Outputs\parcel_pts_geog.csv")
building_data = pd.read_csv(r"E:\Tasks\Replace_HAFB_Parcels\Base_Year_Data\buildings.csv")
zoning_parcels_p = pd.read_csv(r"E:\Tasks\Replace_HAFB_Parcels\Base_Year_Data\zoning_parcels_p.csv")
zoning_baseline = pd.read_csv(r"E:\Tasks\Replace_HAFB_Parcels\Base_Year_Data\zoning_baseline.csv")
jobs = pd.read_csv(r"E:\Tasks\Replace_HAFB_Parcels\Base_Year_Data\jobs.csv")
households = pd.read_csv(r"E:\Tasks\Replace_HAFB_Parcels\Base_Year_Data\households.csv")

In [21]:
parcels_to_replace = pd.DataFrame.spatial.from_featureclass(r"E:\Tasks\Replace_HAFB_Parcels\Inputs\parcels_2019.gdb\Parcels_To_Delete")
old_ids = parcels_to_replace['parcel_id_REMM'].to_list()

In [34]:
new_parcels = pd.DataFrame.spatial.from_featureclass(r"E:\Tasks\Replace_HAFB_Parcels\Inputs\parcels_2019.gdb\new_hill_afb_parcels")
new_ids = parcels_to_replace['parcel_id_REMM'].to_list()

In [7]:
len(old_ids)

76

# parcels

In [16]:
parcel_data.columns

Index(['parcel_id_REMM', 'CO_NAME', 'COUNTY_ID', 'PARCEL_ID',
       'TOTAL_MKT_VALUE', 'land_value', 'residential_units', 'building_sqft',
       'FLOORS_CNT', 'year_built', 'EFFBUILT_YR', 'IS_OUG', 'max_dua',
       'max_far', 'max_height', 'type1', 'type2', 'type3', 'type4', 'type5',
       'type6', 'type7', 'type8', 'agriculture', 'basebldg', 'NoBuild',
       'redev_friction', 'building_type_id', 'x', 'y', 'parcel_acres', 'Split',
       'Split_Factor', 'parcel_id_REMM_old', 'building_type',
       'residential_price', 'non_residential_price', 'non_residential_sqft',
       'TAZID_832', 'TAZID_900', 'Old_PID', 'parent_parcel', 'elevation',
       'fwy_exit', 'airport', 'rail_depot', 'stream', 'trail', 'university',
       'shape_area', 'volume_one_way', 'volume_two_way', 'airport_distance',
       'fwy_exit_dist', 'raildepot_dist', 'university_dist', 'trail_dist',
       'stream_dist', 'train_station', 'rail_stn_dist', 'bus_rte_dist',
       'bus_stop', 'bus_stop_dist', 'volume_tw

In [15]:
#get max parcel id for setting new ids
parcel_data['parcel_id_REMM'].max()

766011

In [76]:
old_parcels = parcel_data[parcel_data['parcel_id_REMM'].isin(old_ids)==True]
del old_parcels['type1']
del old_parcels['type2']
del old_parcels['type3']
del old_parcels['type4']
del old_parcels['type5']
del old_parcels['type6']
del old_parcels['type7']
del old_parcels['type8']
del old_parcels['max_dua']
del old_parcels['max_far']
del old_parcels['max_height']
del old_parcels['CO_NAME']
del old_parcels['COUNTY_ID']
del old_parcels['residential_units']
del old_parcels['building_sqft']
del old_parcels['building_type_id']
del old_parcels['non_residential_sqft']
del old_parcels['year_built']
old_parcels

,parcel_id_REMM,PARCEL_ID,TOTAL_MKT_VALUE,land_value,FLOORS_CNT,EFFBUILT_YR,IS_OUG,agriculture,basebldg,NoBuild,redev_friction,x,y,parcel_acres,Split,Split_Factor,parcel_id_REMM_old,building_type,residential_price,non_residential_price,TAZID_832,TAZID_900,Old_PID,parent_parcel,elevation,fwy_exit,airport,rail_depot,stream,trail,university,shape_area,volume_one_way,volume_two_way,airport_distance,fwy_exit_dist,raildepot_dist,university_dist,trail_dist,stream_dist,train_station,rail_stn_dist,bus_rte_dist,bus_stop,bus_stop_dist,volume_two_way_nofwy,distsml_id,distmed_id,distlrg_id,zonal_ppa
58,59,550020101.0,18173933.0,624958.0,0.75,NaN,0.0,0.0,1.0,0.0,NaN,1.498691e+06,7.578768e+06,2.749255,0,1.0,59,Governmental,NaN,17548975.0,477.0,631.0,456148,456148,1447.911911,0.0,0.0,1.0,0.0,0.0,0.0,1.197575e+05,183.0,364.0,1.80304,0.64824,0.05292,5.79389,0.98743,0.13638,0.0,2.200447,245.827116,0.0,0.758438,364.0,103,12,4,56943.864914
900,949,90120001.0,5983950.0,5983950.0,0.00,NaN,0.0,0.0,1.0,0.0,NaN,1.512920e+06,7.573747e+06,395.572784,0,1.0,949,Governmental,NaN,0.0,478.0,629.0,453810,453810,1447.911911,0.0,1.0,0.0,0.0,0.0,0.0,1.723115e+07,9817.0,19250.0,1.09601,2.09126,2.69915,5.68831,1.11974,0.01841,0.0,2.925626,1331.030000,0.0,0.762553,19250.0,103,12,4,67179.397642
903,952,90150001.0,8110785.0,8110785.0,0.00,NaN,0.0,0.0,1.0,0.0,NaN,1.508639e+06,7.573781e+06,630.888977,0,1.0,952,Governmental,NaN,0.0,480.0,634.0,376765,376765,1447.911911,0.0,1.0,1.0,0.0,0.0,0.0,2.748152e+07,3336.0,6674.0,0.79104,1.44407,1.97201,5.87974,0.43981,0.12520,0.0,2.211474,56.114181,0.0,0.567468,6674.0,103,12,4,57083.205451
904,953,90150002.0,87723.0,87723.0,0.00,NaN,0.0,0.0,1.0,0.0,NaN,1.506058e+06,7.571251e+06,0.504058,0,1.0,953,Governmental,NaN,0.0,480.0,655.0,376870,376870,1447.911911,0.0,0.0,0.0,0.0,0.0,0.0,2.195679e+04,12543.0,25292.0,1.38332,0.87283,1.92523,6.49713,0.42305,0.22079,0.0,1.546994,420.890836,0.0,0.095293,25292.0,103,12,4,57083.205451
905,954,90160001.0,5659050.0,5659050.0,0.00,NaN,0.0,0.0,1.0,0.0,NaN,1.503378e+06,7.575107e+06,376.746216,0,1.0,954,Governmental,NaN,0.0,480.0,633.0,456146,456146,1447.911911,0.0,1.0,1.0,0.0,0.0,0.0,1.641107e+07,13279.0,26611.0,1.16737,0.79185,1.10976,6.06358,0.48624,0.03828,0.0,1.657693,905.424456,0.0,0.892410,26611.0,103,12,4,57083.205451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13980,14725,91410003.0,79050.0,79050.0,0.00,0.0,0.0,0.0,1.0,0.0,NaN,1.496756e+06,7.591363e+06,3.874170,0,1.0,14725,Governmental,NaN,0.0,399.0,527.0,511306,511306,1373.018907,0.0,0.0,0.0,0.0,0.0,0.0,1.687588e+05,0.0,0.0,2.53801,0.21808,2.36910,4.64813,1.24011,0.00764,0.0,2.335740,475.762746,0.0,0.236632,477.0,85,10,3,63434.476681
13981,14726,91420001.0,28050.0,28050.0,0.00,0.0,0.0,0.0,1.0,0.0,NaN,1.496677e+06,7.590968e+06,1.620177,0,1.0,14726,Governmental,NaN,0.0,399.0,527.0,470967,470967,1381.652998,0.0,0.0,0.0,0.0,0.0,0.0,7.057493e+04,0.0,0.0,2.60968,0.28291,2.30482,4.69569,1.22616,0.01047,0.0,2.395132,437.579395,0.0,0.248121,477.0,85,10,3,63434.476681
13982,14727,91420002.0,255000.0,255000.0,0.00,0.0,0.0,0.0,1.0,0.0,NaN,1.496562e+06,7.590251e+06,15.221557,0,1.0,14727,Governmental,NaN,0.0,399.0,527.0,471990,471990,1381.652998,0.0,0.0,0.0,0.0,0.0,0.0,6.630510e+05,0.0,0.0,2.75131,0.41969,2.17545,4.78572,1.15391,0.00698,0.0,2.518513,318.226901,0.0,0.218033,28425.0,85,10,3,63434.476681
459095,462172,NaN,121792000.0,6547308.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.502634e+06,7.572472e+06,244.667007,0,1.0,462172,Multi Family Res,115244692.0,NaN,479.0,655.0,817000,384008,1408.718501,0.0,0.0,1.0,0.0,0.0,0.0,1.065769e+07,3381.0,6784.0,1.63254,0.29004,1.33569,6.56346,0.06569,0.00949,0.0,1.155412,412.970863,0.0,0.761326,6784.0,103,12,4,77718.396893


# Buildings and jobs

In [44]:
jobs

,jobs_id,building_id,cid,sector_id
0,0,27383,35,1
1,1,27383,35,1
2,2,27383,35,1
3,3,27383,35,1
4,4,27383,35,1
...,...,...,...,...
1009643,1009643,18972,57,10
1009644,1009644,625504,49,10
1009645,1009645,625526,49,10
1009646,1009646,4012,11,10


In [46]:
buildings_old_parcels = building_data[building_data['parcel_id'].isin(old_ids)==True]
buildings_old_id_list = buildings_old_parcels['building_id'].to_list()
jobs_old_parcels = jobs[jobs['building_id'].isin(buildings_old_id_list)==True]
jobs_old_parcels['sector_id'].value_counts()

3    2278
6     277
7      57
4      47
9      42
1      31
Name: sector_id, dtype: int64

In [56]:
jobs_by_building = jobs_old_parcels.pivot_table(index='building_id',columns='sector_id',values='jobs_id',aggfunc='count',fill_value=0).reset_index()
jobs_by_building.columns = ['building_id','sector1','sector3','sector4','sector6','sector7','sector9']
jobs_by_building

,building_id,sector1,sector3,sector4,sector6,sector7,sector9
0,59,2,215,0,0,0,0
1,958,0,1,0,0,0,0
2,1281,0,0,0,98,0,0
3,1373,2,186,0,0,0,0
4,1376,2,533,0,0,0,0
5,1380,6,0,0,31,0,0
6,1387,2,1041,45,71,37,42
7,1388,5,0,0,0,0,0
8,1389,6,0,0,29,0,0
9,1390,5,0,0,48,20,0


In [73]:
buildings_to_join = buildings_old_parcels.merge(jobs_by_building, on='building_id', how='left').fillna(0)
buildings_to_join.head()

,building_id,building_sqft,building_type_id,non_residential_sqft,note,parcel_id,residential_units,stories,unit_price_non_residential,year_built,res_price_per_sqft,job_spaces,sector1,sector3,sector4,sector6,sector7,sector9
0,59,65796.0,6.0,65796.0,base,59,0.0,1.0,0.0,2014.0,0.0,217.0,2.0,215.0,0.0,0.0,0.0,0.0
1,949,0.0,6.0,0.0,base,949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,952,7444850.0,6.0,7444850.0,base,952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,953,0.0,6.0,0.0,base,953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,954,0.0,6.0,0.0,base,954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
# join old parcels and old buildings
old_parcels_buildings = old_parcels.merge(buildings_to_join, left_on='parcel_id_REMM', right_on='parcel_id', how='left')
old_parcels_buildings.shape

(76, 68)

In [78]:
parcels_to_replace = parcels_to_replace[['parcel_id_REMM', 'SHAPE']].copy()
parcels_to_replace.shape

(76, 2)

In [79]:
old_parcels_buildings2 = old_parcels_buildings.merge(parcels_to_replace, left_on='parcel_id_REMM', right_on='parcel_id_REMM', how='left')
old_parcels_buildings2.spatial.to_featureclass(location=os.path.join(gdb, 'old_parcels_buildings'), sanitize_columns=False)

'e:\\Tasks\\Replace_HAFB_Parcels\\Outputs\\scratch.gdb\\old_parcels_buildings'

# Process Results

In [264]:
parcel_to_add1 = pd.DataFrame.spatial.from_featureclass(r"E:\Tasks\Replace_HAFB_Parcels\Inputs\parcels_2019.gdb\hafb_parcels_to_add")
parcel_to_add2 = pd.DataFrame.spatial.from_featureclass(r"E:\Tasks\Replace_HAFB_Parcels\Inputs\parcels_2019.gdb\new_hill_afb_parcels")
original_parcels = pd.DataFrame.spatial.from_featureclass(r"E:\Tasks\Replace_HAFB_Parcels\Inputs\remm_base_year_2019.gdb\parcels_2019")

In [86]:
parcel_data_for_insertion = parcel_data[~parcel_data['parcel_id_REMM'].isin(old_ids)==True].copy()
original_parcels_for_insertion = original_parcels[~original_parcels['parcel_id_REMM'].isin(old_ids)==True].copy()

In [83]:
original_parcels_for_insertion.spatial.to_featureclass(location=os.path.join(gdb, 'original_parcels_for_insertion'), sanitize_columns=False)

c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


'e:\\Tasks\\Replace_HAFB_Parcels\\Outputs\\scratch.gdb\\original_parcels_for_insertion'

In [116]:
del parcel_to_add1['OBJECTID']
del parcel_to_add2['OBJECTID']
del parcel_to_add2['SHAPE']

In [117]:

parcel_to_add = parcel_to_add1.merge(parcel_to_add2, on='parcel_id_REMM', how='inner')
parcel_to_add.columns = ['parcel_id_REMM', 'job_spaces', 'TOTAL_MKT_VALUE',
       'land_value', 'residential_price', 'non_residential_price',
       'building_sqft', 'non_residential_sqft',
       'residential_units', 'unit_price_non_residential', 'jobs1',
       'jobs2', 'jobs3', 'jobs4', 'jobs5', 'jobs6', 'jobs7', 'jobs8', 'jobs9',
       'SHAPE', 'TAZID', 'parcel_acres', 'CO_NAME', 'COUNTY_ID', 'PARCEL_ID',
       'building_type_id', 'building_type', 'TAZID_832', 'TAZID_900',
       'max_dua', 'max_far', 'type1', 'type2', 'type3', 'type4', 'type5',
       'type6', 'redev_friction', 'max_height', 'type7', 'type8', 'year_built','IS_OUG', 'NoBuild','shape_area']

parcel_to_add.spatial.to_featureclass(location=os.path.join(r'E:\Tasks\Replace_HAFB_Parcels\Inputs\parcels_2019.gdb', 'parcel_to_add'), sanitize_columns=False)

'E:\\Tasks\\Replace_HAFB_Parcels\\Inputs\\parcels_2019.gdb\\parcel_to_add'

In [163]:
parcels_to_replace_for_pts = parcels_to_replace[['parcel_id_REMM','SHAPE']].copy().merge(parcel_data[['parcel_id_REMM', 'elevation', 'x','y','fwy_exit','airport','rail_depot',	'stream','trail','university',	
                                                                         'volume_one_way','volume_two_way','airport_distance','fwy_exit_dist','raildepot_dist',	'university_dist',	
                                                                         'trail_dist','stream_dist','train_station','rail_stn_dist','bus_rte_dist','bus_stop',	'bus_stop_dist',	
                                                                         'volume_two_way_nofwy','distsml_id','distmed_id','distlrg_id','zonal_ppa']].copy(), on='parcel_id_REMM', how = 'left')

parcels_to_replace_for_pts.spatial.to_featureclass(location=os.path.join(gdb, 'parcels_to_replace_for_pts'), sanitize_columns=False)                                                            

'e:\\Tasks\\Replace_HAFB_Parcels\\Outputs\\scratch.gdb\\parcels_to_replace_for_pts'

In [164]:
arcpy.FeatureToPoint_management(os.path.join(gdb, 'parcels_to_replace_for_pts'), os.path.join(gdb, 'parcels_2015_attributes_pts'), "INSIDE")
arcpy.FeatureToPoint_management(os.path.join(r'E:\Tasks\Replace_HAFB_Parcels\Inputs\parcels_2019.gdb', 'parcel_to_add'), os.path.join(gdb, 'parcel_to_add_pts'), "INSIDE")

<Result '.\\Outputs\\scratch.gdb\\parcel_to_add_pts'>

In [165]:
parcel_to_add3 = pd.DataFrame.spatial.from_featureclass(r"E:\Tasks\Replace_HAFB_Parcels\Inputs\parcels_2019.gdb\new_parcels_2015_attributes_pts")
del parcel_to_add3['OBJECTID']
del parcel_to_add3['SHAPE']
parcels_to_add = parcel_to_add.merge(parcel_to_add3, on='parcel_id_REMM', how='inner')

In [121]:
parcels_to_add.columns

Index(['parcel_id_REMM', 'job_spaces', 'TOTAL_MKT_VALUE', 'land_value',
       'residential_price', 'non_residential_price', 'building_sqft',
       'non_residential_sqft', 'residential_units',
       'unit_price_non_residential', 'jobs1', 'jobs2', 'jobs3', 'jobs4',
       'jobs5', 'jobs6', 'jobs7', 'jobs8', 'jobs9', 'SHAPE', 'TAZID',
       'parcel_acres', 'CO_NAME', 'COUNTY_ID', 'PARCEL_ID', 'building_type_id',
       'building_type', 'TAZID_832', 'TAZID_900', 'max_dua', 'max_far',
       'type1', 'type2', 'type3', 'type4', 'type5', 'type6', 'redev_friction',
       'max_height', 'type7', 'type8', 'year_built', 'IS_OUG', 'NoBuild',
       'shape_area', 'Join_Count', 'TARGET_FID', 'elevation', 'fwy_exit',
       'airport', 'rail_depot', 'stream', 'trail', 'university',
       'volume_one_way', 'volume_two_way', 'airport_distance', 'fwy_exit_dist',
       'raildepot_dist', 'university_dist', 'trail_dist', 'stream_dist',
       'train_station', 'rail_stn_dist', 'bus_rte_dist', 'bus_stop

In [84]:
parcel_data_for_insertion.head()

,parcel_id_REMM,CO_NAME,COUNTY_ID,PARCEL_ID,TOTAL_MKT_VALUE,land_value,residential_units,building_sqft,FLOORS_CNT,year_built,EFFBUILT_YR,IS_OUG,max_dua,max_far,max_height,type1,type2,type3,type4,type5,type6,type7,type8,agriculture,basebldg,NoBuild,redev_friction,building_type_id,x,y,parcel_acres,Split,Split_Factor,parcel_id_REMM_old,building_type,residential_price,non_residential_price,non_residential_sqft,TAZID_832,TAZID_900,Old_PID,parent_parcel,elevation,fwy_exit,airport,rail_depot,stream,trail,university,shape_area,volume_one_way,volume_two_way,airport_distance,fwy_exit_dist,raildepot_dist,university_dist,trail_dist,stream_dist,train_station,rail_stn_dist,bus_rte_dist,bus_stop,bus_stop_dist,volume_two_way_nofwy,distsml_id,distmed_id,distlrg_id,zonal_ppa
0,1,Davis,11,31870103.0,2221724.0,716332.0,0.0,37890.0,1.692308,1932.0,NaN,NaN,25.0,NaN,NaN,f,t,f,t,t,f,t,f,0.0,1.0,0.0,NaN,2.0,1.535281e+06,7.493206e+06,0.309205,0,1.0,1,Multi Family Res,1505392.0,NaN,NaN,611.0,832.0,456950,456950,1337.899530,0.0,0.0,0.0,0.0,0.0,0.0,13468.983890,2285.0,4599.0,8.64950,0.86466,4.61749,8.93864,1.29588,0.04215,0.0,1.321618,52.459794,0.0,0.057695,1839.0,140,16,5,216907.881909
1,2,Davis,11,102660101.0,32801790.0,2405375.0,0.0,858460.0,1.857143,2005.0,NaN,0.0,30.0,NaN,NaN,f,t,f,f,f,f,t,f,0.0,1.0,0.0,NaN,6.0,1.507989e+06,7.564025e+06,6.417644,0,1.0,2,Governmental,NaN,30396415.0,858460.0,527.0,704.0,439032,439032,1363.717932,0.0,0.0,0.0,0.0,0.0,0.0,279552.573681,48495.0,48495.0,2.65535,0.68337,2.90702,7.69982,0.92118,0.59113,0.0,1.816734,1145.970000,0.0,0.355620,10976.0,117,14,4,375684.023663
2,3,Davis,11,NaN,NaN,NaN,0.0,17550.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,f,f,f,t,t,f,t,f,0.0,1.0,NaN,NaN,3.0,1.521750e+06,7.486432e+06,1.212642,0,1.0,3,Industrial,NaN,NaN,17550.0,620.0,867.0,456419,456419,1292.263429,0.0,0.0,0.0,0.0,0.0,0.0,52822.704306,3798.0,7140.0,6.26761,0.97995,3.01113,8.75806,0.43739,0.00999,0.0,1.542745,3524.010000,0.0,0.669830,7140.0,143,16,5,127830.094924
3,4,Davis,11,NaN,NaN,NaN,0.0,19794.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,f,f,f,t,t,f,t,f,0.0,1.0,NaN,NaN,3.0,1.521639e+06,7.488655e+06,1.061592,0,1.0,4,Industrial,NaN,NaN,19794.0,619.0,862.0,457530,457530,1292.263429,0.0,0.0,0.0,0.0,0.0,0.0,46242.946773,2453.0,5337.0,6.64955,0.61776,3.42522,9.12498,0.39910,0.20578,0.0,1.413702,5722.080000,0.0,1.088381,5337.0,143,16,5,111011.593316
4,5,Davis,11,NaN,NaN,NaN,0.0,73650.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,f,f,t,f,f,f,f,f,0.0,1.0,NaN,NaN,3.0,1.532196e+06,7.495228e+06,1.128954,0,1.0,5,Industrial,NaN,NaN,73650.0,605.0,825.0,418768,418768,1314.335335,0.0,0.0,0.0,0.0,0.0,0.0,49177.254124,9205.0,16804.0,8.64670,0.25125,4.77342,9.45593,0.73060,0.04612,0.0,1.119122,871.842463,0.0,0.324787,16804.0,139,16,5,238073.073945


In [166]:
parcel_to_add_for_csv = parcels_to_add[['parcel_id_REMM', 'CO_NAME', 'COUNTY_ID', 'PARCEL_ID','TOTAL_MKT_VALUE', 'land_value','residential_units','building_sqft','year_built','IS_OUG',
                                        'max_dua', 'max_far', 'type1', 'type2', 'type3', 'type4', 'type5', 'type6','type7', 'type8', 'redev_friction', 'max_height' , 'NoBuild',
                                        'building_type_id', 'x','y', 'parcel_acres','residential_price', 'non_residential_price','non_residential_sqft','TAZID_832', 'TAZID_900', 'building_type',
                                        'elevation', 'fwy_exit', 'airport', 'rail_depot',
                                        'stream', 'trail', 'university','shape_area', 'volume_one_way', 'volume_two_way',
                                        'airport_distance', 'fwy_exit_dist', 'raildepot_dist',
                                        'university_dist', 'trail_dist', 'stream_dist', 'train_station',
                                        'rail_stn_dist', 'bus_rte_dist', 'bus_stop', 'bus_stop_dist',
                                        'volume_two_way_nofwy', 'distsml_id', 'distmed_id', 'distlrg_id',
                                        'zonal_ppa', 'SHAPE']].copy()


parcel_to_add_for_shape = parcel_to_add_for_csv.copy()

del parcel_to_add_for_csv['SHAPE']

In [152]:
parcel_data_for_insertion.columns.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [158]:
parcel_to_add_for_csv


In [224]:
# export to csv
new_parcels_2019_csv = pd.concat([parcel_data_for_insertion.reset_index(drop=True), parcel_to_add_for_csv.reset_index(drop=True)])


In [225]:
# fix tazud
# del parcel_geog['OID_']
parcel_geog.columns = ['parcel_id_REMM','new_TAZID_832', 'FIPS']
new_parcels_2019_csv = new_parcels_2019_csv.merge(parcel_geog, on='parcel_id_REMM', how='left')
new_parcels_2019_csv['TAZID_832'] = new_parcels_2019_csv['new_TAZID_832']
new_parcels_2019_csv['COUNTY_ID'] = new_parcels_2019_csv['FIPS']

del new_parcels_2019_csv['new_TAZID_832']
del new_parcels_2019_csv['FIPS']

In [226]:
new_parcels_2019_csv.to_csv(os.path.join(outputs[0], 'parcels_20220513.csv'))   

In [131]:
original_parcels_for_insertion.head()

,OBJECTID,parcel_id_REMM,CO_NAME,TAZID,COUNTY_ID,PARCEL_ID,TOTAL_MKT_VALUE,land_value,residential_units,building_sqft,FLOORS_CNT,year_built,EFFBUILT_YR,IS_OUG,max_dua,max_far,max_height,type1,type2,type3,type4,type5,type6,type7,type8,agriculture,basebldg,NoBuild,redev_friction,building_type_id,x,y,parcel_acres,Split,Split_Factor,parcel_id_REMM_old,building_type,residential_price,non_residential_price,non_residential_sqft,TAZID_832,TAZID_900,SHAPE
0,1,1,Davis,None,11,031870103,2221724.0,716332.0,0.0,37890.0,1.692308,1932.0,None,None,25.0,None,None,f,t,f,t,t,f,t,f,0.0,1.0,0.0,None,2.0,425844.34375,4526921.5,0.309205,0,1.0,1,Multi Family Res,1505392.0,NaN,NaN,611.0,832.0,"{""rings"": [[[425865.8903999999, 4526947.611400..."
1,2,2,Davis,None,11,102660101,32801790.0,2405375.0,0.0,858460.0,1.857143,2005.0,None,0,30.0,None,None,f,t,f,f,f,f,t,f,0.0,1.0,0.0,None,6.0,417525.96875,4548699.0,6.417644,0,1.0,2,Governmental,NaN,30396415.0,858460.0,527.0,704.0,"{""rings"": [[[417720.6777999997, 4548575.704700..."
2,3,3,Davis,None,11,None,NaN,NaN,0.0,NaN,NaN,NaN,None,0,0.0,None,None,f,f,f,t,t,f,t,f,NaN,1.0,NaN,None,3.0,NaN,NaN,1.212642,0,1.0,3,Industrial,NaN,NaN,NaN,620.0,867.0,"{""rings"": [[[421778.1858999999, 4524869.0756],..."
3,4,4,Davis,None,11,None,NaN,NaN,0.0,NaN,NaN,NaN,None,0,0.0,None,None,f,f,f,t,t,f,t,f,NaN,1.0,NaN,None,3.0,NaN,NaN,1.061592,0,1.0,4,Industrial,NaN,NaN,NaN,619.0,862.0,"{""rings"": [[[421713.7997000003, 4525534.078600..."
4,5,5,Davis,None,11,None,NaN,NaN,0.0,NaN,NaN,NaN,None,0,0.0,None,None,f,f,t,f,f,f,f,f,NaN,1.0,NaN,None,3.0,NaN,NaN,1.128954,0,1.0,5,Industrial,NaN,NaN,NaN,605.0,825.0,"{""rings"": [[[424969.12870000023, 4527569.08510..."


In [267]:
parcels_to_add_for_shape = parcel_to_add_for_shape[['parcel_id_REMM','CO_NAME','COUNTY_ID','PARCEL_ID','TOTAL_MKT_VALUE','land_value','residential_units','building_sqft',
	                                               'year_built','IS_OUG','max_dua','max_far','max_height','type1','type2','type3','type4','type5','type6','type7','type8',
                                                   'NoBuild','redev_friction','building_type_id','x','y','parcel_acres','building_type','residential_price','non_residential_price',	
                                                   'non_residential_sqft','TAZID_832','TAZID_900','SHAPE']].copy()


In [268]:
parcels_to_add_for_shape.columns

Index(['parcel_id_REMM', 'CO_NAME', 'COUNTY_ID', 'PARCEL_ID',
       'TOTAL_MKT_VALUE', 'land_value', 'residential_units', 'building_sqft',
       'year_built', 'IS_OUG', 'max_dua', 'max_far', 'max_height', 'type1',
       'type2', 'type3', 'type4', 'type5', 'type6', 'type7', 'type8',
       'NoBuild', 'redev_friction', 'building_type_id', 'x', 'y',
       'parcel_acres', 'building_type', 'residential_price',
       'non_residential_price', 'non_residential_sqft', 'TAZID_832',
       'TAZID_900', 'SHAPE'],
      dtype='object')

In [270]:
new_parcels_2019 = pd.concat([original_parcels_for_insertion, parcels_to_add_for_shape])


In [271]:
new_parcels_2019 = new_parcels_2019.merge(parcel_geog, on='parcel_id_REMM', how='left')
new_parcels_2019['TAZID_832'] = new_parcels_2019['new_TAZID_832']
new_parcels_2019['COUNTY_ID'] = new_parcels_2019['FIPS']

del new_parcels_2019['new_TAZID_832']
del new_parcels_2019['FIPS']

In [272]:
new_parcels_2019.spatial.to_featureclass(location=os.path.join(gdb2, 'parcels_2019'), sanitize_columns=False)   

'e:\\Tasks\\Replace_HAFB_Parcels\\Outputs\\remm_base_year.gdb\\parcels_2019'

# buildings

In [239]:
b= pd.DataFrame.spatial.from_featureclass(os.path.join(r'E:\Tasks\Replace_HAFB_Parcels\Inputs\parcels_2019.gdb', 'parcel_to_add'))

new_buildings =  b[['parcel_id_REMM', 'building_sqft', 'building_type_id', 'non_residential_sqft','residential_price','job_spaces','unit_price_non_residential' ,'residential_units','year_built']].copy()
new_buildings['res_price_per_sqft'] = new_buildings['residential_price']/new_buildings['building_sqft']
new_buildings['stories'] = 1
new_buildings['note'] = 'base'
new_buildings = new_buildings.fillna(0)
new_buildings['building_id'] = new_buildings['parcel_id_REMM']
new_buildings['parcel_id'] = new_buildings['parcel_id_REMM']
del new_buildings['residential_price']
del new_buildings['parcel_id_REMM']

In [240]:
new_buildings

,building_sqft,building_type_id,non_residential_sqft,job_spaces,unit_price_non_residential,residential_units,year_built,res_price_per_sqft,stories,note,building_id,parcel_id
0,7.224415e+02,6,7.224415e+02,0.0,0.0,0.0,1940,0.000000,1,base,766399,766399
1,7.249185e+00,6,7.249185e+00,0.0,0.0,0.0,1940,0.000000,1,base,766402,766402
2,1.560476e+04,6,1.560476e+04,0.0,0.0,0.0,1940,0.000000,1,base,766474,766474
3,1.461023e+05,6,1.461023e+05,0.0,0.0,0.0,1940,0.000000,1,base,766475,766475
4,5.213498e+04,6,3.972330e+04,0.0,0.0,0.0,1940,0.000000,1,base,766476,766476
5,5.689157e+05,6,5.689157e+05,0.0,0.0,0.0,1940,0.000000,1,base,766477,766477
6,3.076048e+05,6,3.076048e+05,0.0,0.0,0.0,1940,0.000000,1,base,766478,766478
7,1.032106e+06,2,1.324789e+04,0.0,0.0,1128.0,1953,184.796838,1,base,766479,766479
8,7.200952e+06,6,7.187898e+06,0.0,0.0,0.0,1940,0.000000,1,base,766480,766480


In [263]:
buildings_for_insertion = building_data[~building_data['parcel_id'].isin(old_ids)==True].copy()
new_buildings_2019_csv = pd.concat([buildings_for_insertion, new_buildings])
new_buildings_2019_csv.to_csv(os.path.join(outputs[0],'buildings_20220513.csv'), index=False)

# households

In [175]:
households.loc[households['building_id'].isin([462172,462173]) == True, 'building_id'] = 766479
households.to_csv(os.path.join(outputs[0],'households_20220513.csv'))

# jobs

In [186]:
def convert(df, col, sector_id):
    df = df.reindex(df.index.repeat(df[col]))[['building_id', 'cid']].copy()
    df['sector_id'] = sector_id
    return df

# # create individual job table
# cols = ['jobs1','jobs2','jobs3','jobs4','jobs5','jobs6','jobs7','jobs8','jobs9','jobs10','jobs11']
# f = pd.concat([convert(df,c, int(c.split("s",1)[1])) for c in cols]).reset_index(drop=True)
# f['jobs_id'] = f.index
# f = f[['jobs_id','building_id','cid','sector_id']].copy()
# f.to_csv(r'.\outputs\jobs.csv', index=False)

In [256]:
buildings = parcel_to_add1[['parcel_id_REMM', 'jobs1','jobs3','jobs4','jobs5','jobs6', 'jobs7','jobs9']].copy()
buildings['cid'] = 11
buildings.rename({'parcel_id_REMM':'building_id'}, axis=1, inplace=True)

In [184]:
# buildings2 = buildings.reindex(buildings.index.repeat(buildings['jobs1']))[['building_id', 'cid']].copy()
# buildings2['sector_id'] = 1
# buildings2

,building_id,cid,sector_id
4,766476,11,1
4,766476,11,1
4,766476,11,1
4,766476,11,1
4,766476,11,1
...,...,...,...
8,766480,11,1
8,766480,11,1
8,766480,11,1
8,766480,11,1


In [258]:
cols = ['jobs1','jobs3','jobs4','jobs5','jobs6','jobs7','jobs9']
f = pd.concat([convert(buildings,c, int(c.split("s",1)[1])) for c in cols]).reset_index(drop=True)
f['jobs_id'] = f.index
f = f[['jobs_id','building_id','cid','sector_id']].copy()
# f.to_csv(r'.\outputs\jobs.csv', index=False)
newjobs = f

In [259]:
jobs_for_insertion = jobs[~jobs['building_id'].isin(old_ids)==True].copy()
new_jobs_2019_csv = pd.concat([jobs_for_insertion, newjobs])


In [260]:
new_jobs_2019_csv = new_jobs_2019_csv.merge(parcel_geog, left_on='building_id', right_on='parcel_id_REMM', how='left')
new_jobs_2019_csv['cid'] = new_jobs_2019_csv['FIPS']

del new_jobs_2019_csv['FIPS']
del new_jobs_2019_csv['new_TAZID_832']
del new_jobs_2019_csv['parcel_id_REMM']

In [262]:
new_jobs_2019_csv.to_csv(os.path.join(outputs[0], 'jobs_20220513.csv'), index=False)   

# zoning baseline

In [41]:
zoning_baseline_without_old_parcels = zoning_baseline[~zoning_baseline['parcel_id'].isin(old_ids)==True]

In [36]:
zoning_baseline_new_parcels = new_parcels[['parcel_id_REMM','max_dua', 'max_far', 'max_height', 'type1','type2','type3','type4','type5','type6','type7','type8']].copy()
zoning_baseline_new_parcels.rename({'parcel_id_REMM':'parcel_id'}, axis=1, inplace=True)
zoning_baseline_new_parcels

,parcel_id,max_dua,max_far,max_height,type1,type2,type3,type4,type5,type6,type7,type8
0,766399,0.0,0.0,999.0,f,f,f,f,f,f,f,f
1,766402,0.0,0.0,999.0,f,f,f,f,f,f,f,f
2,766474,0.0,0.0,999.0,f,f,f,f,f,f,f,f
3,766475,0.0,0.0,999.0,f,f,f,f,f,f,f,f
4,766476,0.0,0.0,999.0,f,f,f,f,f,f,f,f
5,766477,0.0,0.0,999.0,f,f,f,f,f,f,f,f
6,766478,0.0,0.0,999.0,f,f,f,f,f,f,f,f
7,766479,0.0,0.0,999.0,f,f,f,f,f,f,f,f
8,766480,0.0,0.0,999.0,f,f,f,f,f,f,f,f


In [216]:
new_zoning_baseline = pd.concat([zoning_baseline_without_old_parcels, zoning_baseline_new_parcels])
new_zoning_baseline
new_zoning_baseline.to_csv(os.path.join(outputs[0],'zoning_baseline_20220513.csv.csv'))

# zoning parcels p

In [251]:
zoning_parcels_p = zoning_parcels_p[~zoning_parcels_p['parcel_id'].isin(old_ids)==True].copy()
zoning_parcels_p.to_csv(os.path.join(outputs[0],'zoning_parcels_p_20220513.csv'), index=False)

In [246]:
pdts2015 = pd.read_csv(r"E:\REMM\Base_Year_Data_2019\v1\parceldistance2transitstops_2015.csv")
pdts2022 = pd.read_csv(r"E:\REMM\Base_Year_Data_2019\v1\parceldistance2transitstops_2022.csv")
pdts2027 = pd.read_csv(r"E:\REMM\Base_Year_Data_2019\v1\parceldistance2transitstops_2027.csv")
pdts2035 = pd.read_csv(r"E:\REMM\Base_Year_Data_2019\v1\parceldistance2transitstops_2035.csv")
pdts2045 = pd.read_csv(r"E:\REMM\Base_Year_Data_2019\v1\parceldistance2transitstops_2045.csv")

pdts2015 = pdts2015[~pdts2015['parcel_id'].isin(old_ids)==True].copy()
pdts2022 = pdts2022[~pdts2022['parcel_id'].isin(old_ids)==True].copy()
pdts2027 = pdts2027[~pdts2027['parcel_id'].isin(old_ids)==True].copy()
pdts2035 = pdts2035[~pdts2035['parcel_id'].isin(old_ids)==True].copy()
pdts2045 = pdts2045[~pdts2045['parcel_id'].isin(old_ids)==True].copy()

rows = parcel_to_add_for_csv[['parcel_id_REMM']].copy()
rows['dist2busst'] = 5253
rows['dist2rails'] = 9150
rows

,parcel_id_REMM,dist2busst,dist2rails
0,766399,5253,9150
1,766402,5253,9150
2,766474,5253,9150
3,766475,5253,9150
4,766476,5253,9150
5,766477,5253,9150
6,766478,5253,9150
7,766479,5253,9150
8,766480,5253,9150


In [252]:
pdts2015 = pd.concat([pdts2015, rows])
pdts2015.to_csv(r".\Outputs\parceldistance2transitstops_2015_20220513.csv", index=False)

pdts2022 = pd.concat([pdts2022, rows])
pdts2022.to_csv(r".\Outputs\parceldistance2transitstops_2022_20220513.csv", index=False)

pdts2027 = pd.concat([pdts2027, rows])
pdts2027.to_csv(r".\Outputs\parceldistance2transitstops_2025_20220513.csv",index=False)

pdts2035 = pd.concat([pdts2035, rows])
pdts2035.to_csv(r".\Outputs\parceldistance2transitstops_2035_20220513.csv",index=False)

pdts2045 = pd.concat([pdts2045, rows])
pdts2045.to_csv(r".\Outputs\parceldistance2transitstops_2045_20220513.csv", index=False)

In [255]:
rf = new_parcels_2019_csv[['parcel_id_REMM', 'redev_friction']].copy()
rf.to_csv(r".\Outputs\redev_friction_20220513.csv",index=False)